# Initial Requirements

In [1]:
# Mount your google drive to the colab environment
from google.colab import drive
drive.mount('/content/gdrive')

# copy data
# !cp '/content/gdrive/My Drive/Colab Notebooks/Twitter_Analysis/OLD_tweets_with_hashtag_AIDebate.txt' before_debate.txt
# !cp '/content/gdrive/My Drive/Colab Notebooks/Twitter_Analysis/tweets_with_hashtag_AIDebate1.txt' after_debate.txt




# change enviroment variable of keras's backend to theano
import os; os.environ['KERAS_BACKEND'] = 'theano'

# get and install emotion predictor model
!git clone https://github.com/nikicc/twitter-emotion-recognition.git 
!ls # list of files

# go to the directory
%cd twitter-emotion-recognition/

# install required versions of the libraries
!pip install -r requirements.txt





# install library for language detection
!pip install langdetect

# download NLTK packages
import nltk
!python -m nltk.downloader all

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Cloning into 'twitter-emotion-recognition'...
remote: Enumerating objects: 54, done.
remote: Total 54 (delta 0), reused 0 (delta 0), pack-reused 54
Unpacking objects: 100% (54/54), done.
gdrive	sample_data  twitter-emotion-recognition
/content/twitter-emotion-recognition
     |████████████████████████████████| 153kB 2.7MB/s 
     |████████████████████████████████| 2.9MB 46.5MB/s 
  Created wheel for keras: filename=Keras-1.1.0-cp36-none-any.whl size=178686 sh

# Emotion Analysis Part

In [2]:



# analyze each tweet
import datetime
from emotion_predictor import EmotionPredictor



class TweetEmotion:
  
  # load emotion prediction model
  def __init__(self):
    self.model = EmotionPredictor(classification='ekman', setting='mc')


  # what emotion?
  def what_emotion(self, tweet):
    prediction = self.model.predict_classes([tweet])
    # print('The associated emotion with [' + tweet + '] seems to be:' + '\n' + str(prediction))
    return prediction

  # probability of emotion
  def probability_of_emotion(self, tweet):
    probability = self.model.predict_probabilities([tweet])
    # print('The associated probablity of emotion with [' + tweet + '] seems to be:' + '\n' + str(probability))
    # return [probability['Anger'], probability['Disgust'], probability['Fear'],probability['Joy'], probability['Sadness'], probability['Surprise']]
    p = pd.Series({'Anger': float(probability['Anger']),'Disgust': float(probability['Disgust']), 'Fear': float(probability['Fear']), 'Joy': float(probability['Joy']), 'Sadness': float(probability['Sadness']),  'Surprise': float(probability['Surprise'])})
    return p



# tweet_emotion = TweetEmotion()






Using Theano backend.


# Tweet Cleaning and Sentiment Analysis

In [0]:
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import langdetect

def sentiment_and_freq(tweet):

  # check whether tweet's language is English
  is_english = False
  lang = langdetect.detect(tweet)
  if(lang != 'en'):
    is_english = False
    # print("non-English")
  else:
    # print("Yeah, English")
    is_english = True


  # Clean the Tweet

  # split into words
  tokens = word_tokenize(tweet)
  # convert to lower case
  tokens = [w.lower() for w in tokens]
  # remove punctuation from each word
  import string
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in tokens]
  # remove remaining tokens that are not alphabetic
  words = [word for word in stripped if word.isalpha()]
  # filter out stop words
  from nltk.corpus import stopwords
  stop_words = set(stopwords.words('english'))
  words = [w for w in words if not w in stop_words]




  # detokenize to calculate sentiment polarity
  cleaned_sentence = TreebankWordDetokenizer().detokenize(words)
  polarity_of_tweet = TextBlob(cleaned_sentence).sentiment.polarity
  # print(polarity_of_tweet)





  # count specific words (speakers' first or family names)
  yoshua_name = ['yoshua' , 'bengio', 'yoshuabengio']
  gary_name = ['gary' , 'marcus', 'garymarcus']

  has_yoshua = any(word in cleaned_sentence for word in yoshua_name)
  has_gary = any(word in cleaned_sentence for word in gary_name)


    # count specific words (deep learning, symbol, hybrid)
  deeplearning = ['deeplearning' , 'deep learning', 'dl', 'deep_learning']
  symbol = ['symbol' , 'symbols', 'symbolic']
  hybrid = ['hybrid']

  has_deeplearning = any(word in cleaned_sentence for word in deeplearning)
  has_symbol = any(word in cleaned_sentence for word in symbol)
  has_hybrid = any(word in cleaned_sentence for word in hybrid)


  a = pd.Series({'is_english': is_english, 'polarity_of_tweet':polarity_of_tweet, 'has_yoshua':has_yoshua, 'has_gary':has_gary, 
                 'has_deeplearning':has_deeplearning, 'has_symbol':has_symbol, 'has_hybrid':has_hybrid})
  return a




# Analyze All Tweets

In [4]:
import json
import pandas as pd
# import TweetEmotion

# csv file of the data
file = '/content/gdrive/My Drive/Colab Notebooks/Twitter_Analysis/ALL_tweets_with_hashtag_AIDebate.csv'

# read as pandas dataframe
df = pd.read_csv(file)

# standardize the time
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')

# initialize emotion prediction model
tweet_emotion = TweetEmotion()

# determine emotion
df[['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']] = df['text'].apply(tweet_emotion.probability_of_emotion)

# determine language of tweet, analyze its sentiment, and frequency of speakers' names
df[['is_english',  'polarity_of_tweet',  'has_yoshua',  'has_gary', 'has_deeplearning', 'has_symbol', 'has_hybrid']] = df['text'].apply(sentiment_and_freq)
                     

print(df['date'])


WARNING (theano.tensor.blas): We did not found a dynamic library into the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


0     2019-12-31 17:27:05
1     2019-12-31 17:11:53
2     2019-12-31 17:04:32
3     2019-12-31 14:36:41
4     2019-12-31 14:00:51
              ...        
451   2019-12-17 16:10:34
452   2019-12-17 02:11:19
453   2019-12-17 00:14:05
454   2019-12-16 13:46:20
455   2019-12-16 07:31:11
Name: date, Length: 456, dtype: datetime64[ns]


# Save the Results

In [0]:
save_path = '/content/gdrive/My Drive/Colab Notebooks/Twitter_Analysis/Analyzed_tweets.csv'
df.to_csv(save_path)